# Import, Init

In [138]:
import os, sys
import pandas as pd
import seaborn as sns

sys.path.insert(0, "./classes/")
import cls_nnetwork

# reloading functions without runtime.restart
import importlib
importlib.reload(cls_nnetwork)

<module 'cls_nnetwork' from './classes/cls_nnetwork.py'>

In [139]:
# cut -- train test data with trimmed columns, so that it's more easy to train

nn_manager_cut = cls_nnetwork.NeuralManager(dir_path='../data/csv_to_train/train_test_cut')

>>> train-test inited:  {'X_test_cut': True, 'X_train_cut': True, 'y_test_cut': True, 'y_train_cut': True}


In [140]:
nn_manager_cut.X_train.shape, nn_manager_cut.X_test.shape

((3182, 12), (795, 12))

In [141]:
nn_manager_cut.X_train[:5]

,Open,High,Low,Vol,AVBLS,NTRAT,TOUTV,TRFEE,BTC_MINED_PDAY,VOL_CHANGE_PDAY,MWNUS_CH_PDAY,NTRAT_CH_PDAY
Date,,,,,,,,,,,,
2019-04-03,4859.3,5278.4,4814.0,13830000.0,1.248330,397961949.0,1.306792e+06,142.102988,425.0,-12780000.0,52351.0,360333.0
2019-04-02,4145.1,4899.6,4143.5,14770000.0,1.215507,397560778.0,3.429199e+06,114.489789,1512.5,-940000.0,61873.0,401171.0
2019-04-01,4102.3,4159.1,4076.8,3220000.0,1.205358,397191600.0,1.744093e+06,61.846737,2162.5,11550000.0,87465.0,369178.0
2019-03-31,4111.8,4121.9,4082.2,2430000.0,1.173566,396808790.0,9.168389e+05,44.614255,1425.0,790000.0,34688.0,382810.0
2019-03-30,4103.7,4138.1,4057.1,2550000.0,1.204920,396430531.0,9.228557e+05,54.673729,2050.0,-120000.0,64109.0,378259.0


# Model Combining

In [142]:
from tensorflow.keras.layers import TimeDistributed 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense

n_steps = 4
n_features = nn_manager_cut.X_train.shape[1]
n_seq = 2
n_steps = 2
conv1D_params = dict(filters=64, kernel_size=1, activation='relu')

template = [
    TimeDistributed(Conv1D(**conv1D_params), input_shape=(None, n_steps, n_features)),
    TimeDistributed(MaxPooling1D(pool_size=2)),
    TimeDistributed(Flatten()),
    LSTM(50, activation='relu'),
    Dense(1)
]

In [143]:
nn_manager_cut.model_combine(template=template)

>>> model compiled
Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_60 (TimeDis (None, None, 2, 64)       832       
_________________________________________________________________
time_distributed_61 (TimeDis (None, None, 1, 64)       0         
_________________________________________________________________
time_distributed_62 (TimeDis (None, None, 64)          0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 50)                23000     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 51        
Total params: 23,883
Trainable params: 23,883
Non-trainable params: 0
_________________________________________________________________


True

In [145]:
x_train = nn_manager_cut.X_train, y_train = nn_manager_cut.y_train
nn_manager_cut.model.fit(
    x=x_train, y=y_train,
    epochs= 5,
#     batch_size = 32,
    validation_data = (nn_manager_cut.X_test, nn_manager_cut.y_test),
    verbose = 2
)

Epoch 1/5


ValueError: in user code:

    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:869 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:859 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:852 run_step  **
        outputs = model.t